# Estimador de precios de inmuebles en la ciudad de San Juan, Argentina.

## 1. Inicialización: librerías, configuraciones y variables globales

### Importación de Librerías

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import re
import unicodedata
import os
from datetime import datetime, timedelta

## 2. Importación de Base de datos

### Buscar la base de datos

In [ ]:

conn = sqlite3.connect('inmuebles_full.db')
df_all = pd.read_sql_query("SELECT * FROM inmuebles", conn)
conn.close()

## 3. Normalización de datos

In [ ]:
df_all.loc[df_all['precio'] == "Consultar", 'precio'] = None
df_all['moneda'] = df_all['precio'].str.extract(r'([^\d\s]+)')
df_all['precio'] = df_all['precio'].str.extract(r'(\d[\d,.]*)')
df_all['precio'] = df_all['precio'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

df_all['visitas'] = df_all['visitas'].str.replace('.', '').astype(int)
df_all['anuncio_id'] = df_all['anuncio_id'].astype(int)
df_all['superficie_total'] = df_all['superficie_total'].str.extract(r'(\d+)').astype(float)
df_all['superficie_cubierta'] = df_all['superficie_cubierta'].str.extract(r'(\d+)').astype(float)
antiguedad_dummies = pd.get_dummies(df_all['antiguedad'], prefix='antiguedad')
df_all = pd.concat([df_all, antiguedad_dummies], axis=1)
df_all.drop(columns=['antiguedad'], inplace=True)

df_all['apto_credito'] = df_all['apto_credito'].map({'SI': True, 'NO': False})
df_all['apto_credito'] = df_all['apto_credito'].astype(bool)
df_all['en_barrio_privado'] = df_all['en_barrio_privado'].map({'SI': True, 'NO': False})
df_all['en_barrio_privado'] = df_all['en_barrio_privado'].astype(bool)

# Crear dummies para la columna 'departamento'
departamento_dummies = pd.get_dummies(df_all['departamento'], prefix='departamento')

# Concatenar los dummies al dataframe original
df_all = pd.concat([df_all, departamento_dummies], axis=1)

# Eliminar la columna original 'departamento'
df_all.drop(['departamento'], axis=1, inplace=True)

# Crear dummies para las columnas banos, ambientes, garages, dormitorios
banos_dummies = pd.get_dummies(df_all['banos'], prefix='banos')
ambientes_dummies = pd.get_dummies(df_all['ambientes'], prefix='ambientes')
garages_dummies = pd.get_dummies(df_all['garages'], prefix='garages')
dormitorios_dummies = pd.get_dummies(df_all['dormitorios'], prefix='dormitorios')

# Concatenar los dummies al dataframe original
df_all = pd.concat([df_all, banos_dummies, ambientes_dummies, garages_dummies, dormitorios_dummies], axis=1)

# Eliminar las columnas originales
df_all.drop(['banos', 'ambientes', 'garages', 'dormitorios'], axis=1, inplace=True)


# Crear dummies para la columna 'orientacion'
orientacion_dummies = pd.get_dummies(df_all['orientacion'], prefix='orientacion')

# Concatenar los dummies al dataframe original
df_all = pd.concat([df_all, orientacion_dummies], axis=1)

# Eliminar la columna original 'orientacion'
df_all.drop(['orientacion'], axis=1, inplace=True)

# Borrar la columna 'procesado' si existe
df_all.drop(['procesado'], axis=1, inplace=True)

In [ ]:
# Fecha de extracción
fecha_extraccion = datetime(2025, 3, 25)

# Función para convertir valores no correspondientes a fechas
def convertir_a_fecha(valor):
    if valor == 'Hoy':
        return fecha_extraccion.strftime('%d/%m/%Y')
    elif valor == 'Ayer':
        return (fecha_extraccion - timedelta(days=1)).strftime('%d/%m/%Y')
    elif re.match(r'Hace (\d+) días', valor):
        dias = int(re.search(r'(\d+)', valor).group(1))
        return (fecha_extraccion - timedelta(days=dias)).strftime('%d/%m/%Y')
    else:
        return valor

# Aplicar la función a la columna 'publicacion'
df_all['publicacion'] = df_all['publicacion'].apply(convertir_a_fecha)

# Convertir la columna 'publicacion' a tipo datetime
df_all['publicacion'] = pd.to_datetime(df_all['publicacion'], format='%d/%m/%Y', errors='coerce')


In [ ]:
# Eliminar registros sin longitud y latitud
df_all = df_all.dropna(subset=['latitude', 'longitude'])

# Eliminar las columnas especificadas
df_all = df_all.drop(columns=['calle', 'barrio', 'numero'])


In [ ]:
response = requests.get("https://dolarapi.com/v1/dolares/bolsa")
dolar_bolsa_data = response.json()

datos_dolar = {
    "moneda": dolar_bolsa_data.get("moneda"),
    "casa": dolar_bolsa_data.get("casa"),
    "nombre": dolar_bolsa_data.get("nombre"),
    "compra": dolar_bolsa_data.get("compra"),
    "venta": dolar_bolsa_data.get("venta"),
    "fechaActualizacion": dolar_bolsa_data.get("fechaActualizacion")
}

valor_dolar = (datos_dolar['compra'] + datos_dolar['venta']) / 2

# cambiar los precios con moneda en pesos a dólares
df_all.loc[df_all['moneda'] == '$', 'precio'] = df_all['precio'] / valor_dolar
df_all.drop(columns=['moneda'], inplace=True)


In [ ]:
# Configurar pandas para mostrar todas las columnas y filas sin recortar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Mostrar los tipos de datos del DataFrame
df_all.dtypes

In [ ]:
# Concatenar las columnas 'comentario_caracteristicas', 'comentario' y 'titulo', manejando valores nulos
df_all['comentarios'] = df_all[['comentario_caracteristicas', 'comentario', 'titulo']].fillna('').agg(' '.join, axis=1)

# Eliminar las columnas originales 'comentario_caracteristicas', 'comentario' y 'titulo'
df_all.drop(columns=['comentario_caracteristicas', 'comentario', 'titulo'], inplace=True)

# df_all['comentarios'] = df_all['comentarios'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Mostrar todos los comentarios
print(df_all['comentarios'])


In [ ]:
df_all.head()

In [ ]:
conn = sqlite3.connect('inmuebles_normalizada.db')
df_all.to_sql('inmuebles', conn, if_exists='replace', index=False)
conn.close()